## Setup

In [24]:
%pdb on
import sys
!which python
!python --version

Automatic pdb calling has been turned ON
/Users/Fango/anaconda3/envs/RAG-env/bin/python
Python 3.11.14


In [66]:
import importlib
import time
from typing import List
import logging, sys
import nest_asyncio
import gradio as gr

from sentence_transformers import CrossEncoder

from llama_index.core import Document
from llama_index.core import SimpleDirectoryReader
from llama_index.core import Settings
from llama_index.core import SummaryIndex, VectorStoreIndex

from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.node_parser import TokenTextSplitter
from llama_index.core.postprocessor import SentenceTransformerRerank
from llama_index.core.schema import NodeWithScore
from llama_index.core.tools import QueryEngineTool
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector

# Adapter for talking to a local LLM served by Ollama (http://localhost:11434).
from llama_index.llms.ollama import Ollama

# Adapter to call a local embedding model via Ollama (e.g., nomic-embed-text).
from llama_index.embeddings.ollama import OllamaEmbedding



In [26]:
##logger = logging.getLogger("llama_index")
##logger.setLevel(logging.DEBUG)
##handler = logging.StreamHandler(sys.stdout)
##handler.setLevel(logging.DEBUG)
##logger.addHandler(handler)

nest_asyncio.apply()

In [27]:
# 1) LLM + embeddings (local via Ollama)
# Creates an LLM handle pointing at your local Ollama model.
# - model: the exact model tag pulled with `ollama pull ...`
# - request_timeout: upper bound for long answers/slow laptops.
#Settings.llm = Ollama(model="qwen2.5:7b-instruct-q4_K_M", embed_batch = 10, request_timeout=600)
Settings.llm = Ollama(model="aya:8b-23-q4_K_M", embed_batch = 10, request_timeout=600)
#Settings.llm = Ollama(model="mixtral:latest", embed_batch = 10, request_timeout=600)

# Creates an embedding function that turns text → vectors using Ollama.
# Using a dedicated embedding model improves retrieval quality versus using the LLM.
Settings.embed_model = OllamaEmbedding(model_name="nomic-embed-text:latest",embed_batch_size=8,request_timeout=600,)

In [28]:
### Old code
# load documents

documents = SimpleDirectoryReader(input_files=["CR-hospitalisation-24092025.pdf"]).load_data()

#print(len(documents))               # how many docs?
#print(len(documents[0].text))       # length of first doc
#print(documents[0].text[:2000])     # first 1000 chars

In [72]:

### new code with file upload feature
all_nodes: List = []  # global container for *all* chunks from all files

def split_and_embed_documents(documents: List[Document]) -> int:
    """
    Take a list of Documents, split them with your TokenTextSplitter logic,
    compute embeddings, and extend the global `all_nodes`.
    Returns the number of new nodes added.
    """
    short_splitter  = TokenTextSplitter(chunk_size=150, chunk_overlap=20)
    medium_splitter = TokenTextSplitter(chunk_size=250, chunk_overlap=40)
    long_splitter   = TokenTextSplitter(chunk_size=400, chunk_overlap=60)
    
    global all_nodes

    new_nodes = []

    for doc in documents:
        n_tokens = len(doc.text.split())  # quick estimate

        if n_tokens < 400:
            nodes = short_splitter.get_nodes_from_documents([doc])
        elif n_tokens < 4000:
            nodes = medium_splitter.get_nodes_from_documents([doc])
        else:
            nodes = long_splitter.get_nodes_from_documents([doc])

        # Add metadata and embeddings
        for n in nodes:
            n.metadata["source_doc_id"] = doc.metadata.get("file_name", "unknown")
            n.embedding = Settings.embed_model.get_text_embedding(n.get_content())

        new_nodes.extend(nodes)

    # Add to the global list
    all_nodes.extend(new_nodes)

    return len(new_nodes)


summary_index = None
vector_index = None
query_engine = None

def rebuild_indexes():
    global summary_index, vector_index, query_engine

    if not all_nodes:
        summary_index = None
        vector_index = None
        query_engine = None
        return

    summary_index = SummaryIndex(all_nodes)
    vector_index = VectorStoreIndex(all_nodes)

    # Choose whatever you actually use in your RAG:
    #query_engine = vector_index.as_query_engine(response_mode="compact")

In [73]:
# Split + embed + fill all_nodes
n_nodes = split_and_embed_documents(documents)

# Build indexes and query_engine from all_nodes
rebuild_indexes()

print(f"Index initial : {len(documents)} document(s), {n_nodes} chunks.")

> Adding chunk: ASSISTANCE
PUBLIQUE
HÔPITAUX
DE PARIS
Hôpital
P...


2025-11-05 20:32:02,760 - DEBUG - > Adding chunk: ASSISTANCE
PUBLIQUE
HÔPITAUX
DE PARIS
Hôpital
P...


> Adding chunk: Université
Service des Maladies Infectieuses et...


2025-11-05 20:32:02,762 - DEBUG - > Adding chunk: Université
Service des Maladies Infectieuses et...


> Adding chunk: Romain PALICH
Dr Cécile POUDEROUX
Dr Sophie SEA...


...

> Adding chunk: de produits sanguins labiles : Non
- Administra...


2025-11-05 20:32:04,650 - DEBUG - > Adding chunk: de produits sanguins labiles : Non
- Administra...


> Adding chunk: Jacques LEFEBVRE| M| 13/03/1947| INS/INIR: 1470...


2025-11-05 20:32:04,651 - DEBUG - > Adding chunk: Jacques LEFEBVRE| M| 13/03/1947| INS/INIR: 1470...
2025-11-05 20:32:04,672 - INFO - HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
2025-11-05 20:32:04,697 - INFO - HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
2025-11-05 20:32:04,727 - INFO - HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
2025-11-05 20:32:04,749 - INFO - HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
2025-11-05 20:32:04,772 - INFO - HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
2025-11-05 20:32:04,797 - INFO - HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
2025-11-05 20:32:04,823 - INFO - HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"


Index initial : 7 document(s), 63 chunks.


In [48]:

### Old code

short_splitter  = TokenTextSplitter(chunk_size=150, chunk_overlap=20)
medium_splitter = TokenTextSplitter(chunk_size=250, chunk_overlap=40)
long_splitter   = TokenTextSplitter(chunk_size=400, chunk_overlap=60)

all_nodes = []

for doc in documents:
    n_tokens = len(doc.text.split())  # quick-and-dirty estimate

    if n_tokens < 400:          # short doc → no or tiny splitting
        nodes = short_splitter.get_nodes_from_documents([doc])
    elif n_tokens < 4000:       # medium
        nodes = medium_splitter.get_nodes_from_documents([doc])
    else:                       # long doc → bigger chunks, more overlap
        nodes = long_splitter.get_nodes_from_documents([doc])

    # Optionally add a doc_type or source metadata here
    for n in nodes:
        n.metadata["source_doc_id"] = doc.metadata.get("file_name", "unknown")

    all_nodes.extend(nodes)

#lengths = [len(n.get_content()) for n in all_nodes]
#print(f"Chunks: {len(lengths)}  |  avg size: {sum(lengths)//len(lengths)} chars  |  max: {max(lengths)}")

for node in all_nodes:
    node.embedding = Settings.embed_model.get_text_embedding(node.get_content())

> Adding chunk: ASSISTANCE
PUBLIQUE
HÔPITAUX
DE PARIS
Hôpital
P...


2025-11-05 19:33:03,292 - DEBUG - > Adding chunk: ASSISTANCE
PUBLIQUE
HÔPITAUX
DE PARIS
Hôpital
P...


> Adding chunk: Université
Service des Maladies Infectieuses et...


2025-11-05 19:33:03,293 - DEBUG - > Adding chunk: Université
Service des Maladies Infectieuses et...


> Adding chunk: Romain PALICH
Dr Cécile POUDEROUX
Dr Sophie SEA...


...

2025-11-05 19:33:03,365 - DEBUG - > Adding chunk: RARE - FLHERMITTE RDC)
- 28/10/2025 à 16:00: (P...


> Adding chunk: de produits sanguins labiles : Non
- Administra...


2025-11-05 19:33:03,365 - DEBUG - > Adding chunk: de produits sanguins labiles : Non
- Administra...


> Adding chunk: Jacques LEFEBVRE| M| 13/03/1947| INS/INIR: 1470...


2025-11-05 19:33:03,366 - DEBUG - > Adding chunk: Jacques LEFEBVRE| M| 13/03/1947| INS/INIR: 1470...
2025-11-05 19:33:03,408 - INFO - HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
2025-11-05 19:33:03,436 - INFO - HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
2025-11-05 19:33:03,457 - INFO - HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
2025-11-05 19:33:03,480 - INFO - HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
2025-11-05 19:33:03,500 - INFO - HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
2025-11-05 19:33:03,520 - INFO - HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
2025-11-05 19:33:03,545 - INFO - HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
2025-11-05 19:33:03,566 - INFO - HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
2025-11-05 19:33:03,591 - INFO - HTTP Request: POST http://localhost:11434/

In [30]:
### 

summary_index = SummaryIndex(all_nodes)
vector_index = VectorStoreIndex(all_nodes)

In [74]:
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
    similarity_top_k=10,
    #node_postprocessors = [reranker],
)

vector_query_engine = vector_index.as_query_engine(
    similarity_top_k=10,
    #node_postprocessors = [reranker],
)

summary_tool = QueryEngineTool.from_defaults(
    query_engine=summary_query_engine,
    description=(
        "Useful for summarization questions"
    ),
    
)

vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description=(
        "Useful for retrieving specific context."
    ),
)

query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[
        summary_tool,
        vector_tool,
    ],
    verbose=False
)

In [75]:
def ask_rag(question: str) -> str:
    """
    Call your existing RAG pipeline and return plain text.
    Adapt `query_engine` if your variable has another name.
    """
    response = query_engine.query(question)   # <- use your real query engine here
    return str(response)



In [76]:

### New code
def ask_rag(question: str) -> str:
    if query_engine is None:
        return "Aucun document n'est encore indexé."
    response = query_engine.query(question)
    return str(response)

In [77]:

### GUI with ChatInterface API
def add_files_from_upload(files) -> str:
    """
    files: list of gradio.File objects (or None).
    - Load them as Documents
    - Split + embed them with your existing logic
    - Rebuild indexes
    Returns a status string describing what happened.
    """
    if not files:
        return "Aucun fichier fourni."

    paths = [f.name for f in files]

    new_docs = SimpleDirectoryReader(input_files=paths).load_data()
    n_new_nodes = split_and_embed_documents(new_docs)
    rebuild_indexes()

    return f"{len(new_docs)} document(s) ajouté(s), {n_new_nodes} chunk(s) indexé(s)."

def gradio_rag(message, history):
    """
    message: latest user message (string)
    history: list [[user, assistant], ...] – just for display
    """
    answer = ask_rag(message)          # use your RAG pipeline here
    history = history + [[message, answer]]
    return answer

demo = gr.ChatInterface(
    fn=gradio_rag,
    title="Your first local RAG pipeline",
    description="Pose des questions sur tes documents indexés."
)

def on_files_uploaded(files, history):
    """
    Gradio callback when files are uploaded.
    """
    status = add_files_from_upload(files)
    # Add a system-style message to the chat
    history = history + [["[système]", status]]
    return history

/Users/Fango/anaconda3/envs/RAG-env/lib/python3.11/site-packages/gradio/chat_interface.py:347: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(
2025-11-05 20:32:47,373 - INFO - HTTP Request: HEAD https://huggingface.co/api/telemetry/https%3A/api.gradio.app/gradio-initiated-analytics "HTTP/1.1 200 OK"
2025-11-05 20:32:47,926 - INFO - HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"


In [78]:

### Gui with ChatBot API
def rag_chat_stream(message, history):
    """
    Chat callback:
    - Show user message + "…" while RAG runs
    - Then show answer + timer
    """
    if not message:
        yield history, ""
        return

    # Show user message + thinking indicator
    history = history + [[message, "…"]]
    yield history, ""

    t0 = time.time()
    answer = ask_rag(message)
    dt = time.time() - t0

    answer_with_timer = f"{answer}\n\n---\n⏱ Temps de réponse : {dt:.2f} s"
    history[-1][1] = answer_with_timer
    yield history, ""

with gr.Blocks() as demo:
    gr.Markdown("# 💬 RAG Chat (Aya + upload)")

    # 🔹 New: file upload component
    file_upload = gr.File(
        label="Ajouter des fichiers (PDF, TXT, etc.)",
        file_count="multiple"
    )

    chatbot = gr.Chatbot(label="Conversation")
    msg = gr.Textbox(
        label="Votre question",
        placeholder="Pose une question sur tes documents...",
        lines=2,
    )

    with gr.Row():
        send_btn = gr.Button("Envoyer")
        clear_btn = gr.Button("🧹 Effacer")

    # When files are added → update RAG + system message in chat
    file_upload.change(
        fn=on_files_uploaded,
        inputs=[file_upload, chatbot],
        outputs=[chatbot],
    )

    # Enter in textbox
    msg.submit(
        fn=rag_chat_stream,
        inputs=[msg, chatbot],
        outputs=[chatbot, msg],
    )

    # Click on "Envoyer"
    send_btn.click(
        fn=rag_chat_stream,
        inputs=[msg, chatbot],
        outputs=[chatbot, msg],
    )

    # Clear conversation
    clear_btn.click(
        fn=lambda: ([], ""),
        inputs=None,
        outputs=[chatbot, msg],
    )



/var/folders/q0/5c9h5nmd6fq2j1lyzmyc_0gm0000gn/T/ipykernel_9395/2989178719.py:33: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="Conversation")
2025-11-05 20:32:57,061 - INFO - HTTP Request: HEAD https://huggingface.co/api/telemetry/https%3A/api.gradio.app/gradio-initiated-analytics "HTTP/1.1 200 OK"
2025-11-05 20:32:57,585 - INFO - HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"


In [79]:
demo.launch(inline=False)

2025-11-05 20:33:01,514 - INFO - HTTP Request: GET http://127.0.0.1:7862/gradio_api/startup-events "HTTP/1.1 200 OK"
2025-11-05 20:33:01,519 - INFO - HTTP Request: HEAD http://127.0.0.1:7862/ "HTTP/1.1 200 OK"


* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


2025-11-05 20:33:01,627 - INFO - HTTP Request: HEAD https://huggingface.co/api/telemetry/https%3A/api.gradio.app/gradio-launched-telemetry "HTTP/1.1 200 OK"


> Adding chunk: ASSISTANCE
PUBLIQUE
HÔPITAUX
DE PARIS
Hôpital
P...


2025-11-05 20:33:41,941 - DEBUG - > Adding chunk: ASSISTANCE
PUBLIQUE
HÔPITAUX
DE PARIS
Hôpital
P...


...

2025-11-05 20:35:21,602 - DEBUG - > Top 10 nodes:
> [Node 2e8b88a8-78b8-40d0-a91c-a2b8296b89b3] [Similarity score:                 0.718831] Jacques LEFEBVRE| M| 13/03/1947| INS/INIR: 147035917803242 | 8017109544 | 663797412
Imprimé le 29...
> [Node b5c3610a-1e1a-4a06-a143-19e431b7bd1f] [Similarity score:                 0.692668] Jacques LEFEBVRE| M| 13/03/1947 | INS/NIR:147035917803242 | 8017109544 | 663797412
Imprimé le 29/...
> [Node cb0ce39c-d44c-4dee-82e1-4311e0c40afc] [Similarity score:                 0.687192] médical
lsabele GUILLUY (0142.16.02 62)
Assistante sociales
Perrine BUBA (0142. 16.00.86)
Emma AU...
> [Node ed664cf8-70b3-422f-a341-9a67c19702bf] [Similarity score:                 0.676078] kg l/ DFG:90 mL/min
Pat., Jacques LEFEBVRE| M| 13/03/1947 | INSINIR:147036917803242 | 8017109544 ...
> [Node edea15e3-8c4a-43fe-9649-d9f5ae273732] [Similarity score:                 0.666422] D'HOSPITALISATION
Monsieur LEFEBVRE Jacques, né le 13/03/1947, âgé de 78 ans, a été hospita

Selecting query engine 1: The question asks about 'opérations', which could refer to medical operations or procedures. The context suggests that the person in question may have had a medical procedure in 2021, and retrieving specific context related to their health would be useful for answering this question..


2025-11-05 20:37:40,901 - INFO - Selecting query engine 1: The question asks about 'opérations', which could refer to medical operations or procedures. The context suggests that the person in question may have had a medical procedure in 2021, and retrieving specific context related to their health would be useful for answering this question..
2025-11-05 20:37:40,942 - INFO - HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"


> Top 10 nodes:
> [Node eb4419f0-a86b-4e6c-8079-76566a80cde5] [Similarity score:                 0.616002] informaticien/commercial
Autonome dans les actes de la vie quotidienne, pas d'aide à domicile (pe...
> [Node c0b63069-cb1d-449f-aa81-eb0df691cc09] [Similarity score:                 0.615662] Service des Maladies Infectieuses et Tropicales
ASSISTANCE
PUBLIQUE
HÔPITAUX
DE PARIS
Rėcidive su...
> [Node edea15e3-8c4a-43fe-9649-d9f5ae273732] [Similarity score:                 0.611771] D'HOSPITALISATION
Monsieur LEFEBVRE Jacques, né le 13/03/1947, âgé de 78 ans, a été hospitalisé d...
> [Node 2ada9330-637d-4b42-8bdb-957271e25497] [Similarity score:                 0.609228] et coloscopie prévu en externe (le patient sera informé de la date par
mail et courrier)
Plani ﬁ ...
> [Node f895f73e-df79-40ad-ba4a-cfaecb82248f] [Similarity score:                 0.608288] électroporation de ﬁ brillation atriale persistante emboligéne le 13/11/2023 (Dr
DUTHOIT)
•Rėcidi...
> [Node 6de17fab-2ea8-4b

2025-11-05 20:37:40,952 - DEBUG - > Top 10 nodes:
> [Node eb4419f0-a86b-4e6c-8079-76566a80cde5] [Similarity score:                 0.616002] informaticien/commercial
Autonome dans les actes de la vie quotidienne, pas d'aide à domicile (pe...
> [Node c0b63069-cb1d-449f-aa81-eb0df691cc09] [Similarity score:                 0.615662] Service des Maladies Infectieuses et Tropicales
ASSISTANCE
PUBLIQUE
HÔPITAUX
DE PARIS
Rėcidive su...
> [Node edea15e3-8c4a-43fe-9649-d9f5ae273732] [Similarity score:                 0.611771] D'HOSPITALISATION
Monsieur LEFEBVRE Jacques, né le 13/03/1947, âgé de 78 ans, a été hospitalisé d...
> [Node 2ada9330-637d-4b42-8bdb-957271e25497] [Similarity score:                 0.609228] et coloscopie prévu en externe (le patient sera informé de la date par
mail et courrier)
Plani ﬁ ...
> [Node f895f73e-df79-40ad-ba4a-cfaecb82248f] [Similarity score:                 0.608288] électroporation de ﬁ brillation atriale persistante emboligéne le 13/11/2023 (Dr
DUTHOIT)
•

In [80]:
demo.close()

Closing server running on port: 7862
